In [1]:
from pyteomics import mgf
import pandas as pd
import numpy as np
import glob as glob
import os
import sys
import matplotlib.pyplot as plt
import networkx as nx
import pingouin as pg
import seaborn as sns
sys.path.insert(0,'/global/homes/b/bpb/repos/metatlas')
from metatlas.io import feature_tools as ft

import pandas as pd
import numpy as np
import glob as glob
import matplotlib.pyplot as plt
import networkx as nx
import os
import glob

from typing import List, Tuple

from tqdm.notebook import tqdm

# replace with submodules



# sys.path.insert(0,'../')
import analysis_tools as at

INFO:rdkit:Enabling RDKit 2023.09.1 jupyter extensions


In [3]:
    # tolerance in ppm between experimental signal and node mz
mz_ppm_tolerance = 5
peak_height_min = 1e4
num_datapoints_min = 10
# minimum MSMS score 
msms_score_min = 0.5

# minimum MSMS matching ion count
msms_matches_min = 3

# retention time range in minutes for feature finding
rt_range = [1, 700]

# tolerance in daltons used for calculating MS/MS similarity scores
frag_mz_tolerance = 0.05
# collect and merge required data and metadata
node_data = at.graph_to_df()
node_atlas = at.make_node_atlas(node_data, rt_range)
merged_node_data = at.merge_spectral_data(node_data)

params = {'mz_ppm_tolerance':mz_ppm_tolerance,
        'peak_height_min':peak_height_min,
        'num_datapoints_min':num_datapoints_min,
        'msms_score_min':msms_score_min,
        'msms_matches_min':msms_matches_min,
        'rt_range':rt_range,
        'frag_mz_tolerance':frag_mz_tolerance}

INFO:root:Processing original_spectra.mgf
INFO:root:Processing nl_spectra.mgf


(18269, 9)
(18269, 9)


In [16]:
from importlib import reload
reload(at)

<module 'analysis_tools' from '/global/u2/b/bpb/repos/envnet/envnet/use/analysis_tools.py'>

In [18]:
files_data = at.get_files_df(exp_dir,parse_filename=True)
files_data

,filename,experiment,sampletype
0,/global/cfs/cdirs/metatlas/raw_data/egsb/20240...,pDeCo10_20240501,enz-Lac-NA-NA-Quer2mM
1,/global/cfs/cdirs/metatlas/raw_data/egsb/20240...,pDeCo10_20240501,pellet-NA-Brev25uL-NA-NA
2,/global/cfs/cdirs/metatlas/raw_data/egsb/20240...,pDeCo10_20240501,enz-Lac-Brev125uL-CA1mM-Quer1mM
3,/global/cfs/cdirs/metatlas/raw_data/egsb/20240...,pDeCo10_20240501,enz-Lac-Brev125uL-CA1mM-Quer1mM
4,/global/cfs/cdirs/metatlas/raw_data/egsb/20240...,pDeCo10_20240501,enz-Lac-Brev25uL-NA-NA
5,/global/cfs/cdirs/metatlas/raw_data/egsb/20240...,pDeCo10_20240501,pellet-NA-Brev125uL-NA-NA
6,/global/cfs/cdirs/metatlas/raw_data/egsb/20240...,pDeCo10_20240501,enz-Lac-Brev25uL-CA1mM-Quer1mM
7,/global/cfs/cdirs/metatlas/raw_data/egsb/20240...,pDeCo10_20240501,RS-Lac-NA-NA-NA
8,/global/cfs/cdirs/metatlas/raw_data/egsb/20240...,pDeCo10_20240501,enz-Lac-NA-CA2mM-NA
9,/global/cfs/cdirs/metatlas/raw_data/egsb/20240...,pDeCo10_20240501,enz-Lac-NA-CA1mM-Quer1mM


In [19]:
# comparison_file = 'wavestab3_comparisons.txt'
# comparison_file = 'SynDAC5 comparisons.txt' 
# comparison_file = 'WAVESTAB comparisons.txt'
comparison_file = 'SynDac10 Comparisons.txt'
with open(comparison_file) as f:
    comparisons = f.read()
comparisons = comparisons.split('\n\n')
for row in comparisons:
    exec(row)
    # combine all parameters into a single dictionary and export to a file
    mod_params = params.copy()
    mod_params['exp_dir'] = exp_dir
    mod_params['my_groups'] = my_groups
    mod_params['output_filename'] = output_filename

    with open(output_filename.replace('.csv','.params'),'w') as f:
        for key in mod_params.keys():
            f.write("%s: %s\n"%(key,mod_params[key]))

    files_data = at.get_files_df(exp_dir,parse_filename=True,groups=my_groups)
    files = files_data['filename'].tolist()
    # get ms1 and ms2 data
    ms1_data = at.get_sample_ms1_data(node_atlas, files, mz_ppm_tolerance,peak_height_min,num_datapoints_min)
    max_ms1_data = at.get_best_ms1_rawdata(ms1_data,node_data)
    ms2_data = at.get_sample_ms2_data(files,merged_node_data,msms_score_min,msms_matches_min,mz_ppm_tolerance,frag_mz_tolerance)
    ms2_data = pd.concat(ms2_data)
    max_ms2_data = at.get_best_ms2_rawdata(ms2_data)
    best_hits = at.get_best_ms1_ms2_combined(max_ms1_data,max_ms2_data)

    stats_df = at.do_basic_stats(ms1_data,files_data,my_groups)
    output_df = at.make_output_df(node_data,best_hits,stats_df,filename=output_filename)


  0%|          | 0/2 [00:00<?, ?file/s]

  0%|          | 0/2 [00:00<?, ?file/s]

  0%|          | 0/2 [00:00<?, ?file/s]

  0%|          | 0/2 [00:00<?, ?file/s]

  0%|          | 0/2 [00:00<?, ?file/s]

...

  0%|          | 0/3 [00:00<?, ?file/s]

  0%|          | 0/3 [00:00<?, ?file/s]

  0%|          | 0/3 [00:00<?, ?file/s]

  0%|          | 0/3 [00:00<?, ?file/s]

  0%|          | 0/3 [00:00<?, ?file/s]

In [20]:
# get all the files that are "OUTPUT*.csv" and zip them up
files = glob.glob('OUTPUT*.csv')
import zipfile
with zipfile.ZipFile('output_files.zip','w') as z:
    for file in files:
        z.write(file)
        z.write(file.replace('.csv','.params'))
        # os.remove(file)

In [ ]:
import networkx as nx 
G = nx.read_graphml('/global/cfs/cdirs/metatlas/projects/carbon_network/CarbonNetwork.graphml')

# get the first node id
node_id = list(G.nodes())[0]

# get the names of all node attributes
node_attributes = list(G.nodes[node_id].keys())
node_attributes


In [ ]:
df = pd.read_csv('OUTPUT_yezhang-exudates-syncom_Exud-Bd30-1-Syn-vs-Exud-Bd30-1-NoBact.csv')
cols = ['num_datapoints',
       'peak_area', 'peak_height', 'mz_centroid', 'rt_peak',
       'lcmsrun_observed', 'precursor_mz_best_hit', 'ppm_error', 'ms2_node_id',
       'ms2_score', 'ms2_matches', 'ms2_best_match_method',
       'ms2_lcmsrun_observed', 'mean-control', 'mean-treatment',
       'median-control', 'median-treatment', 'standard_error-control',
       'standard_error-treatment', 'std_dev-control', 'std_dev-treatment',
       'p_value', 't_score', 'log2_foldchange']
df['node_id'] = df['node_id'].astype(str)
df.set_index('node_id',inplace=True,drop=True)
df.head()
G = nx.read_graphml('/global/cfs/cdirs/metatlas/projects/carbon_network/CarbonNetwork.graphml')
g = df[cols].to_dict(orient='index')

nx.set_node_attributes(G, g)
nx.write_graphml(G,'carbon_network_withmetatlas_hits-brachy-syncom-yezhang.graphml')